<a href="https://colab.research.google.com/github/AnyaNiks/CatcherInTheRye_Python_Project/blob/main/CatherInTheRye_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF > TXT

In [2]:
!wget https://raw.githubusercontent.com/AnyaNiks/CatcherInTheRye_Python_Project/main/Rye.pdf

--2025-05-23 12:24:21--  https://raw.githubusercontent.com/AnyaNiks/CatcherInTheRye_Python_Project/main/Rye.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-05-23 12:24:21 ERROR 404: Not Found.



In [ ]:
!pip install pymupdf